In [0]:
!pip install pysqlite3

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version_info)

(3, 37, 2)


In [0]:
import pandas as pd
from IPython.display import display, HTML

In [0]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [0]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [0]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table
cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")

# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


In [0]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [0]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [0]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [0]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [0]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [0]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [0]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [0]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [0]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [0]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [0]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [0]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
runSql('Songs shared', qry_sharing)

user_id,song_id
2,1
2,6


In [0]:
#ASSIGNMENT-1. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table
#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time
#4 Generate new recommendations

#5 What are the differences with the static method on #2 above

In [0]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

# First, let's see what recommendations we have from the previous query

cursor.execute("""
WITH song_similarity AS (
    SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
    FROM LISTENS u1
    JOIN LISTENS u2
    ON u1.user_id=u2.user_id
    AND u1.song_id != u2.song_id
    GROUP BY u1.song_id, u2.song_id
    HAVING COUNT(*)>1
),
recs AS (
    SELECT user_id, song2 as song_id, datetime('now') as rec_time
    FROM song_similarity
    JOIN Listens as L
    ON L.song_id = song_similarity.song1
    WHERE song_similarity.song2 NOT IN
    (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
SELECT ROW_NUMBER() OVER (ORDER BY user_id, song_id), rec_time, user_id, song_id
FROM recs;
""")

conn.commit()
runSql('Recommendations Table after insertion', "SELECT * FROM Recommendations;")
conn.close()

recommendation_id,recommendation_time,user_id,song_id
1,2025-09-21 10:18:54,2,1
2,2025-09-21 10:18:54,2,6


In [0]:
qry_minnie_recs = """
SELECT r.recommendation_id, s.title, s.artist, s.genre, r.recommendation_time
FROM Recommendations r
JOIN Songs s ON r.song_id = s.song_id
JOIN Users u ON r.user_id = u.user_id
WHERE u.name = 'Minnie'
ORDER BY r.recommendation_time DESC;
"""

runSql('Recommendations for Minnie', qry_minnie_recs)

recommendation_id,title,artist,genre,recommendation_time
1,Evermore,Taylor Swift,Pop,2025-09-21 10:18:54
2,Yesterday,Beatles,Classic,2025-09-21 10:18:54


In [0]:
# 3. Simplified time-based recommendations that will actually work
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

# Clear existing recommendations
cursor.execute("DELETE FROM Recommendations;")

# Debug: Check the actual timestamp format in your data
runSql('Actual timestamp format in Listens', """
SELECT listen_id, user_id, song_id, rating, listen_time,
       typeof(listen_time) as time_type
FROM Listens 
ORDER BY listen_time DESC NULLS LAST
LIMIT 10;
""")

# Simple time-based approach: recommend songs from users with similar listening history
cursor.execute("""
-- Step 1: Find users who listen to similar songs
WITH user_similarity AS (
    SELECT l1.user_id as user1, l2.user_id as user2,
           COUNT(DISTINCT l1.song_id) as common_songs
    FROM Listens l1
    JOIN Listens l2 ON l1.song_id = l2.song_id
    WHERE l1.user_id != l2.user_id
    GROUP BY l1.user_id, l2.user_id
    HAVING common_songs >= 1
),

-- Step 2: Get songs from similar users that the target user hasn't heard
user_recommendations AS (
    SELECT us.user1 as target_user, l.song_id,
           COUNT(*) as recommendation_strength,
           datetime('now') as rec_time
    FROM user_similarity us
    JOIN Listens l ON us.user2 = l.user_id
    WHERE l.song_id NOT IN (
        SELECT song_id FROM Listens WHERE user_id = us.user1
    )
    GROUP BY us.user1, l.song_id
    HAVING COUNT(*) >= 1
)

-- Step 3: Insert into Recommendations table
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id, song_id)
SELECT 
    ROW_NUMBER() OVER (ORDER BY recommendation_strength DESC, target_user),
    rec_time,
    target_user,
    song_id
FROM user_recommendations;
""")

conn.commit()

# Check results
runSql('Total Recommendations Generated', "SELECT COUNT(*) as total_recommendations FROM Recommendations;")
runSql('All Recommendations', """
SELECT r.recommendation_id, r.recommendation_time, 
       u.user_id, u.name as user_name,
       s.song_id, s.title, s.artist
FROM Recommendations r
JOIN Users u ON r.user_id = u.user_id
JOIN Songs s ON r.song_id = s.song_id
ORDER BY r.recommendation_time DESC, u.name;
""")
conn.close()

listen_id,user_id,song_id,rating,listen_time,time_type
1,1,1,4.5,2024-08-30 14:35:00,text
3,1,6,3.9,2024-08-29 10:15:00,text
5,2,7,4.6,2024-08-28 09:20:00,text
6,2,8,3.9,2024-08-27 16:45:00,text
8,3,2,4.9,2024-08-26 12:30:00,text
2,1,2,4.2,None,null
4,2,2,4.7,None,null
7,3,1,2.9,None,null
9,3,6,NaN,None,null


total_recommendations
6


recommendation_id,recommendation_time,user_id,user_name,song_id,title,artist
5,2025-09-22 13:54:22,3,Daffy,7,Yellow Submarine,Beatles
6,2025-09-22 13:54:22,3,Daffy,8,Hey Jude,Beatles
3,2025-09-22 13:54:22,1,Mickey,7,Yellow Submarine,Beatles
4,2025-09-22 13:54:22,1,Mickey,8,Hey Jude,Beatles
1,2025-09-22 13:54:22,2,Minnie,1,Evermore,Taylor Swift
2,2025-09-22 13:54:22,2,Minnie,6,Yesterday,Beatles


In [0]:
qry_minnie_time_recs = """
SELECT r.recommendation_id, s.title, s.artist, s.genre, 
       r.recommendation_time, u.name as user_name
FROM Recommendations r
JOIN Songs s ON r.song_id = s.song_id
JOIN Users u ON r.user_id = u.user_id
WHERE u.name = 'Minnie'
ORDER BY r.recommendation_time DESC;
"""

runSql('Time-based Recommendations for Minnie', qry_minnie_time_recs)

recommendation_id,title,artist,genre,recommendation_time,user_name
1,Evermore,Taylor Swift,Pop,2025-09-21 10:46:06,Minnie
2,Yesterday,Beatles,Classic,2025-09-21 10:46:06,Minnie


In [0]:

qry_comparison = """
-- Create temporary static recommendations for comparison
WITH static_recommendations AS (
    WITH song_similarity AS (
        SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
        FROM LISTENS u1
        JOIN LISTENS u2 ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
        GROUP BY u1.song_id, u2.song_id
        HAVING COUNT(*) > 0
    ),
    static_recs AS (
        SELECT DISTINCT L.user_id, song_similarity.song2 as song_id, 'Static' as method
        FROM song_similarity
        JOIN Listens as L ON L.song_id = song_similarity.song1
        WHERE song_similarity.song2 NOT IN (
            SELECT song_id FROM Listens as temp WHERE temp.user_id = L.user_id
        )
    )
    SELECT sr.user_id, sr.song_id, sr.method, s.title, s.artist
    FROM static_recs sr
    JOIN Songs s ON sr.song_id = s.song_id
    WHERE sr.user_id = 2  -- Minnie's user_id
),

time_based_recommendations AS (
    SELECT r.user_id, r.song_id, 'Time-based' as method, s.title, s.artist
    FROM Recommendations r
    JOIN Songs s ON r.song_id = s.song_id
    WHERE r.user_id = 2  -- Minnie's user_id
)

-- Combine both methods
SELECT method, song_id, title, artist FROM static_recommendations
UNION ALL
SELECT method, song_id, title, artist FROM time_based_recommendations
ORDER BY method, title;
"""

runSql("Corrected Comparison: Static vs Time-based for Minnie", qry_comparison)

method,song_id,title,artist
Static,1,Evermore,Taylor Swift
Static,6,Yesterday,Beatles
Time-based,1,Evermore,Taylor Swift
Time-based,6,Yesterday,Beatles
